In [4]:
import pandas as pd
import plotly.graph_objs as go
import plotly.express as px

In [5]:
df_circuits = pd.read_csv('./data/circuits.csv', header = 0)
df_drivers = pd.read_csv('./data/drivers.csv',header = 0)
df_races = pd.read_csv('./data/races.csv',header = 0, delimiter = ';')
df_results = pd.read_csv('./data/results.csv',header = 0)
#df_circuits = pd.read_csv('./data/circuits.csv')
df_laps = pd.read_csv('./data/lap_times.csv',header = 0)

df_races.set_index('raceId', inplace=True)
df_results.set_index('resultId', inplace=True)
df_drivers.set_index('driverId', inplace=True)

In [6]:
df_final = pd.read_excel('./data/cdrcr.xlsx')

In [7]:
circuit_name = 'Yas Marina Circuit'

In [197]:
df_points = pd.read_csv('./data/points.csv',header = 0, names=['a', 'b','c','d']).transpose()

In [198]:
df_points

,Year,1st,2nd,3rd,4th,5th,6th,7th,8th,9th,10th
a,1950-1990,8,6,4,3,2,1,-,-,-,-
b,1991-2002,10,6,4,3,2,1,-,-,-,-
c,2003-2009,10,8,6,5,4,3,2,1,-,-
d,2010-Now,25,18,15,12,10,8,6,4,2,1


### Circuit Information

In [8]:
df_finals_circuit =  df_final.loc[(df_final['circuits.name'] == str(circuit_name))]


In [9]:
list(range(1950,2023,5))

[1950,
 1955,
 1960,
 1965,
 1970,
 1975,
 1980,
 1985,
 1990,
 1995,
 2000,
 2005,
 2010,
 2015,
 2020]

In [10]:
df_final_circuit.columns

Index(['resultId', 'raceId', 'driverId', 'positionText', 'positionOrder',
       'races.raceId', 'races.year', 'races.circuitId', 'races.name',
       'drivers.forename', 'drivers.surname', 'drivers.nationality',
       'circuits.name', 'circuits.location', 'circuits.country',
       'circuits.lat', 'circuits.lng', 'constructors.constructorRef',
       'constructors.name', 'constructors.nationality', 'constructorId',
       'number', 'grid', 'position', 'points', 'laps', 'time', 'milliseconds',
       'fastestLap', 'rank', 'fastestLapTime', 'fastestLapSpeed', 'statusId'],
      dtype='object')

In [11]:
df_final_circuit['drivers.fullname'] = df_final_circuit[['drivers.forename','drivers.surname']].apply(lambda x: ' '.join(x), axis=1)

C:\Users\Admin\AppData\Local\Temp/ipykernel_12644/3853478060.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final_circuit['drivers.fullname'] = df_final_circuit[['drivers.forename','drivers.surname']].apply(lambda x: ' '.join(x), axis=1)


In [12]:
city = df_final_circuit['circuits.location'].unique()[0]
country = df_final_circuit['circuits.country'].unique()[0]

In [13]:
best_driver = df_final_circuit[(df_final_circuit['circuits.name'] == circuit_name) & (df_final_circuit['positionText'] == 1)].groupby('drivers.fullname')['drivers.fullname'].count().sort_values(ascending = False).index.tolist()[0]

In [14]:
best_constructor = df_final_circuit[(df_final_circuit['circuits.name'] == circuit_name) & (df_final_circuit['positionText'] == 1)].groupby('constructors.name')['constructors.name'].count().sort_values(ascending = False).index.tolist()[0]

In [15]:
df_final_circuit.columns

Index(['resultId', 'raceId', 'driverId', 'positionText', 'positionOrder',
       'races.raceId', 'races.year', 'races.circuitId', 'races.name',
       'drivers.forename', 'drivers.surname', 'drivers.nationality',
       'circuits.name', 'circuits.location', 'circuits.country',
       'circuits.lat', 'circuits.lng', 'constructors.constructorRef',
       'constructors.name', 'constructors.nationality', 'constructorId',
       'number', 'grid', 'position', 'points', 'laps', 'time', 'milliseconds',
       'fastestLap', 'rank', 'fastestLapTime', 'fastestLapSpeed', 'statusId',
       'drivers.fullname'],
      dtype='object')

In [16]:
king_of_poles = df_final_circuit[(df_final_circuit['circuits.name'] == circuit_name) & (df_final_circuit['grid'] == 1)].groupby('drivers.fullname',)['drivers.fullname'].count().sort_values(ascending = False).index.tolist()[0]

In [17]:
number_ret = len(df_final_circuit[df_final_circuit['positionText'] == 'R'])

In [124]:
year_list = (df_final_circuit['races.year'].unique()).tolist()
year_list.sort()
year_str = str(year_list)[1:-1]
year_str

'2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021'

In [18]:
df_final_circuit[(df_final_circuit['circuits.name'] == circuit_name) & (df_final_circuit['positionText'] == 1)].groupby(['races.year','drivers.fullname'])['drivers.fullname'].count().sort_values(ascending = True)

races.year  drivers.fullname
2009        Sebastian Vettel    1
2010        Sebastian Vettel    1
2011        Lewis Hamilton      1
2012        Kimi Räikkönen      1
2013        Sebastian Vettel    1
2014        Lewis Hamilton      1
2015        Nico Rosberg        1
2016        Lewis Hamilton      1
2017        Valtteri Bottas     1
2018        Lewis Hamilton      1
2019        Lewis Hamilton      1
2020        Max Verstappen      1
2021        Max Verstappen      1
Name: drivers.fullname, dtype: int64

In [19]:
# erro possivel aqui --- averiguar
recent_driver_win, year_win = df_final_circuit[(df_final_circuit['circuits.name'] == circuit_name) & (df_final_circuit['positionText'] == 1)].groupby(['drivers.fullname','races.year'])['drivers.fullname'].count().sort_values(ascending = False).index.tolist()[0]

In [20]:
recent_driver_win, year_win

('Kimi Räikkönen', 2012)

In [126]:
total_races = len(df_final_circuit['races.year'].unique())
pole_is_win = len(df_final_circuit[(df_final_circuit['positionText'] == 1) & (df_final_circuit['grid'] == 1)])

13

In [130]:
number_accidents = len(df_final_circuit[(df_final_circuit['statusId'] == 3) | (df_final_circuit['statusId'] == 4)])


In [136]:
race_with_accident = len(df_final_circuit[(df_final_circuit['statusId'] == 3) | (df_final_circuit['statusId'] == 4)].groupby('races.year').count())

In [137]:
race_with_accident/total_races

0.46153846153846156

### No race result vs finished race

In [21]:
ret_list = ['R', 'F', 'W', 'N', 'D','E']

In [22]:
dvsc_value = 'Constructors'

In [23]:
if dvsc_value == 'Drivers': 
        filter_dc = 'drivers.fullname'
else: filter_dc = 'constructors.name'

In [24]:
plot_percentages = pd.DataFrame(columns=[filter_dc,'dnf','nraces'])

In [25]:
# procurar caterham

In [26]:
dc_n_list = df_final_circuit[filter_dc].unique().tolist()
race_n_list = []
dnf_n_list = []

for dc in dc_n_list: 
    race_count = 0
    dnf_count = 0
    for j, dc_name in enumerate(df_final_circuit[filter_dc]):
        if dc == dc_name: 
            if df_final_circuit['positionText'].iloc[j] in ret_list:
                dnf_count+=1
            race_count+=1

    dnf_n_list.append(dnf_count)
    race_n_list.append(race_count)



In [27]:
plot_percentages[filter_dc] = dc_n_list
plot_percentages.dnf = dnf_n_list
plot_percentages.nraces = race_n_list

In [28]:
plot_percentages

,constructors.name,dnf,nraces
0,Mercedes,2,24
1,Force India,3,20
2,Red Bull,3,26
3,Lotus F1,4,8
4,Renault,3,16
5,Haas F1 Team,1,12
6,Racing Point,2,4
7,McLaren,3,26
8,Toro Rosso,6,22
9,Sauber,1,18


In [29]:
# NÃO ESQUECER QUE PODE HAVER CIRCUITOS SEM DNFS

In [30]:
# não esta a dar ainda 

In [31]:
plot_percentages['dnf_percentage'] = plot_percentages['dnf'] / plot_percentages['nraces']
plot_percentages['nraces_percentage'] = 1-plot_percentages['dnf_percentage']

In [32]:
drivers_client = ['Virgin', 'Red Bull', 'Ferrari']

In [33]:
pplot_percentages = plot_percentages.loc[plot_percentages[filter_dc].isin(drivers_client)]

In [34]:
pplot_percentages

,constructors.name,dnf,nraces,dnf_percentage,nraces_percentage
2,Red Bull,3,26,0.115385,0.884615
15,Virgin,2,4,0.500000,0.500000
18,Ferrari,1,26,0.038462,0.961538


In [35]:
pplot_percentages[filter_dc] = pplot_percentages[filter_dc].astype('str') +':\r\n NºRaces: ' + pplot_percentages['nraces'].astype('str')

C:\Users\Admin\AppData\Local\Temp/ipykernel_12644/3639100850.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pplot_percentages[filter_dc] = pplot_percentages[filter_dc].astype('str') +':\r\n NºRaces: ' + pplot_percentages['nraces'].astype('str')


In [36]:
pplot_percentages

,constructors.name,dnf,nraces,dnf_percentage,nraces_percentage
2,Red Bull:\r\n NºRaces: 26,3,26,0.115385,0.884615
15,Virgin:\r\n NºRaces: 4,2,4,0.500000,0.500000
18,Ferrari:\r\n NºRaces: 26,1,26,0.038462,0.961538


In [37]:
fig = go.Figure()
fig.add_trace(go.Bar(
    y=pplot_percentages[filter_dc],
    x=pplot_percentages['dnf_percentage'],
    name='DNF',
    orientation='h',   
    marker=dict(
        color='rgba(153, 0, 0, 0.6.0)',
        line=dict(color='rgba(0, 0, 0, 1)', width=1)
    )
))
fig.add_trace(go.Bar(
    y=pplot_percentages[filter_dc],
    x=pplot_percentages['nraces_percentage'],
    name='No DNF',
    orientation='h',
    textposition='outside',
    text = pplot_percentages['nraces'],
    marker=dict(
        color='rgba(69, 69, 69, 0.6)',
        line=dict(color='rgba(0, 0, 0, 1)', width=1)
    )
))

fig.update_layout(barmode='stack')
fig.show()

In [38]:
# plot de dnfs por equipa
plot_df_ret = df_final_circuit[(df_final_circuit['circuits.name'] == circuit_name) & (df_final_circuit['positionText'].isin(ret_list))].groupby(filter_dc).count()
plot_df_ret.reset_index(inplace=True)
bad_list = plot_df_ret[filter_dc].unique()
# plot de corridas acabadas por equipa 
plot_df_fin = df_final_circuit[(df_final_circuit['circuits.name'] == circuit_name) & (df_final_circuit[filter_dc].isin(bad_list))].groupby(filter_dc).count()
plot_df_fin.reset_index(inplace=True)

### Lap times

In [39]:
from datetime import time, timedelta

In [40]:
def time_to_mili(s):
    hours, minutes, seconds = (["0", "0"] + s.split(":"))[-3:]
    hours = int(hours)
    minutes = int(minutes)
    seconds = float(seconds)
    miliseconds = int(3600000 * hours + 60000 * minutes + 1000 * seconds)
    return miliseconds

In [41]:
print(time_to_mili('00:01:27.249000'))

87249


### Avg Lap Time 

In [42]:
avg_lap_time = df_final_circuit[df_final_circuit['positionText']==1]
avg_lap_time['avglaptime'] = avg_lap_time['milliseconds'] / avg_lap_time['laps']

C:\Users\Admin\AppData\Local\Temp/ipykernel_12644/965350504.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [96]:
fig = px.scatter(avg_lap_time, x="races.year", y="avglaptime", title='Fastest Lap Time',color_discrete_sequence=px.colors.sequential.RdBu, color="drivers.fullname", symbol="drivers.fullname")
fig.update_layout(font_color = 'white', paper_bgcolor='rgba(0,0,0,0)', yaxis = {'type': 'date','tickformat': '%H:%M:%S:%ss'},
                      plot_bgcolor='rgba(0,0,0,0)', 
                      xaxis_title="Race Year",
                      yaxis_title="Average Lap Time")
fig.update_traces(marker=dict(size=8))
fig.show()

In [44]:
gen_fast_lap = df_final_circuit[df_final_circuit['rank']==1]
gen_fast_lap['fastestLapTime'] = gen_fast_lap['fastestLapTime'].astype('str').apply(lambda x: time_to_mili(x))

C:\Users\Admin\AppData\Local\Temp/ipykernel_12644/3334194387.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



#### Gen Lap

In [45]:
gen_fast_lap = df_final_circuit[df_final_circuit['rank']==1]
gen_fast_lap['fastestLapTime'] = gen_fast_lap['fastestLapTime'].astype('str').apply(lambda x: time_to_mili(x))

C:\Users\Admin\AppData\Local\Temp/ipykernel_12644/3334194387.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [46]:
fig = px.scatter(gen_fast_lap, x="races.year", y="fastestLapTime", title='Fastest Lap Time',color_discrete_sequence=px.colors.sequential.RdBu, color="drivers.fullname", symbol="drivers.fullname")
fig.update_layout(font_color = 'white', paper_bgcolor='rgba(0,0,0,0)', yaxis = {'type': 'date','tickformat': '%H:%M:%S:%ss'},
                      plot_bgcolor='rgba(0,0,0,0)')
fig.show()

In [47]:
number_accidents = len(df_final_circuit[(df_final_circuit['statusId'] == 3) | (df_final_circuit['statusId'] == 4)])


In [48]:
races_list = df_final_circuit['raceId'].unique()

In [49]:
date_list = df_final_circuit['races.year'].unique()

In [50]:
df_laps_circuit = df_laps.loc[(df_laps['raceId'].isin(races_list))]

In [51]:
df_laps_circuit

,raceId,driverId,lap,position,time,milliseconds
21261,858,1,1,1,1:50.984,110984
21262,858,1,2,1,1:47.726,107726
21263,858,1,3,1,1:47.134,107134
21264,858,1,4,1,1:46.964,106964
21265,858,1,5,1,1:46.549,106549
...,...,...,...,...,...,...
514587,1073,847,22,15,1:30.821,90821
514588,1073,847,23,15,1:30.647,90647
514589,1073,847,24,14,1:31.577,91577
514590,1073,847,25,16,1:32.794,92794


In [52]:
df_laps_drivers = df_laps_circuit.groupby(['raceId','driverId'])['milliseconds'].min()

In [53]:
df_laps_drivers.reset_index()

,raceId,driverId,milliseconds
0,17,1,100367
1,17,2,100672
2,17,3,100997
3,17,4,100757
4,17,5,101316
...,...,...,...
264,1073,846,86762
265,1073,847,90647
266,1073,849,89293
267,1073,852,87496


In [54]:
df_fastest_lap = df_laps_drivers.groupby('raceId').min().reset_index()

In [55]:
df_fastest_lap['year'] = date_list

In [56]:
df_fastest_lap

,raceId,milliseconds,year
0,17,100279,2010
1,355,101274,2011
2,858,102612,2012
3,877,103964,2013
4,897,103434,2015
5,918,104496,2018
6,945,104517,2021
7,968,103729,2019
8,988,100650,2017
9,1009,100867,2020


In [57]:
fig = px.line(df_fastest_lap, x="raceId", y="milliseconds", title='Fastest Lap Time', text='raceId')
fig.update_layout(
            yaxis = {'type': 'date','tickformat': '%M:%S:%ss'}
            )
fig.show()

#### Driver Plot

In [60]:
df_final_circuit

,resultId,raceId,driverId,positionText,positionOrder,races.raceId,races.year,races.circuitId,races.name,drivers.forename,...,points,laps,time,milliseconds,fastestLap,rank,fastestLapTime,fastestLapSpeed,statusId,drivers.fullname
6845,20777,355,30,R,23,355,2010,24,Abu Dhabi Grand Prix,Michael,...,0,0,\N,\N,\N,\N,\N,\N,4,Michael Schumacher
7065,20778,355,24,R,24,355,2010,24,Abu Dhabi Grand Prix,Vitantonio,...,0,0,\N,\N,\N,\N,\N,\N,4,Vitantonio Liuzzi
14868,21207,858,20,R,24,858,2011,24,Abu Dhabi Grand Prix,Sebastian,...,0,1,\N,\N,\N,0,\N,\N,29,Sebastian Vettel
14928,21663,877,807,R,24,877,2012,24,Abu Dhabi Grand Prix,Nico,...,0,0,\N,\N,\N,0,\N,\N,4,Nico Hülkenberg
14935,22085,897,8,R,22,897,2013,24,Abu Dhabi Grand Prix,Kimi,...,0,0,\N,\N,\N,0,\N,\N,3,Kimi Räikkönen
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21922,23764,988,20,3,3,988,2017,24,Abu Dhabi Grand Prix,Sebastian,...,15,55,19.33,5673392,5,2,00:01:40.770000,198.416,1,Sebastian Vettel
21942,24184,1009,20,2,2,1009,2018,24,Abu Dhabi Grand Prix,Sebastian,...,18,55,2.581,5982963,54,1,00:01:40.867000,198.225,1,Sebastian Vettel
21958,24610,1030,20,5,5,1030,2019,24,Abu Dhabi Grand Prix,Sebastian,...,10,55,+1:04.357,5710072,55,3,00:01:40.128000,199.688,1,Sebastian Vettel
21997,21185,858,4,2,2,858,2011,24,Abu Dhabi Grand Prix,Fernando,...,18,55,8.457,5840343,41,6,00:01:43.914000,192.412,1,Fernando Alonso


In [67]:
ret_list,dvsc_value,filter_dc,drivers_client


(['R', 'F', 'W', 'N', 'D', 'E'],
 'Constructors',
 'constructors.name',
 ['Virgin', 'Red Bull', 'Ferrari'])

In [98]:
filter_dc = 'drivers.fullname'
drivers_client= ['Felipe Massa', 'Timo Glock', 'Lewis Hamilton']

In [99]:
dc_avg_lap = df_final_circuit[(df_final_circuit[filter_dc].isin(drivers_client))&(df_final_circuit['statusId']==1)]
dc_avg_lap['avglaptime'] = dc_avg_lap['milliseconds'] / dc_avg_lap['laps']

C:\Users\Admin\AppData\Local\Temp/ipykernel_12644/1057471958.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [100]:
dc_avg_lap = dc_avg_lap.groupby(['races.year', filter_dc])['avglaptime'].min().reset_index()

In [101]:
dc_avg_lap

,races.year,drivers.fullname,avglaptime
0,2010,Felipe Massa,109594.636364
1,2010,Lewis Hamilton,108854.527273
2,2011,Felipe Massa,106953.890909
3,2011,Lewis Hamilton,106034.290909
4,2012,Felipe Massa,116028.418182
5,2012,Timo Glock,116638.909091
6,2013,Felipe Massa,108527.127273
7,2013,Lewis Hamilton,108461.327273
8,2014,Felipe Massa,108094.454545
9,2014,Lewis Hamilton,108047.618182


In [102]:
fig = px.line(dc_avg_lap, x="races.year", y="avglaptime", title='Fastest Lap Time',color_discrete_sequence=px.colors.sequential.RdBu, color=filter_dc)
fig.update_layout(font_color = 'white', paper_bgcolor='rgba(0,0,0,0)', yaxis = {'type': 'date','tickformat': '%M:%S:%ss'},
                      #plot_bgcolor='rgba(0,0,0,0)', 
                      xaxis_title="Race Year",
                      yaxis_title="Average Lap Time")
fig.show()


In [ ]:
avg_lap_time = df_final_circuit[df_final_circuit['positionText']==1]
avg_lap_time['avglaptime'] = avg_lap_time['milliseconds'] / avg_lap_time['laps']

### Box Plot 

#### Wins

In [84]:
df_final[(df_final['circuits.name'] == circuit_name) & (df_final['positionText'] == 1)].groupby('drivers.surname')['drivers.surname'].count().sort_values(ascending = False).index.tolist()

['Vettel']

In [103]:
driver_wins = df_final_circuit[df_final_circuit['positionText'] == 1].groupby('drivers.fullname')['resultId'].count().sort_values(ascending = False).reset_index()
driver_wins
fig = px.bar(driver_wins, x='drivers.fullname', y = 'resultId', labels={'drivers.fullname':'Drivers Name', 'resultId': 'Wins'}, color_discrete_sequence=px.colors.sequential.RdBu, text='resultId')
#fig.update_traces(marker_color='rgb(158,202,225)', marker_line_color='rgb(8,48,107)',
#                  marker_line_width=1.5, opacity=0.6)
fig.update_traces(textposition='outside',  texttemplate='<b>%{y} Wins</b>')
fig.update_layout(font_color = 'white', paper_bgcolor='rgba(0,0,0,0)',
                      plot_bgcolor='rgba(0,0,0,0)')
fig.show()


#### Points 

In [45]:
driver_points = df_final_circuit.groupby(['drivers.fullname'])['points'].sum().sort_values(ascending = True).nlargest(15).reset_index()

In [104]:
fig = px.bar(driver_points, x='drivers.fullname', y = 'points', labels={'drivers.fullname':'Drivers Name', 'points': 'Points'}, color_discrete_sequence=px.colors.sequential.RdBu, text='points')
#fig.update_traces(marker_color='rgb(158,202,225)', marker_line_color='rgb(8,48,107)',
#                  marker_line_width=1.5, opacity=0.6)
fig.update_traces(textposition='outside',  texttemplate='<b>%{y} Points</b>')
fig.update_layout(font_color = 'white', paper_bgcolor='rgba(0,0,0,0)',
                      plot_bgcolor='rgba(0,0,0,0)')
fig.show()

### Pie Chart 

In [100]:
df_final.columns

Index(['resultId', 'raceId', 'driverId', 'positionText', 'positionOrder',
       'races.raceId', 'races.year', 'races.circuitId', 'races.name',
       'drivers.forename', 'drivers.surname', 'drivers.nationality',
       'circuits.name', 'circuits.location', 'circuits.country',
       'circuits.lat', 'circuits.lng', 'constructors.constructorRef',
       'constructors.name', 'constructors.nationality', 'constructorId'],
      dtype='object')

In [101]:
const_wins = df_final[(df_final['circuits.name'] == circuit_name) & (df_final['positionText'] == 1)].groupby('constructors.name')['constructors.name'].count().sort_values(ascending = False).tolist()

In [102]:
const_names = df_final[(df_final['circuits.name'] == circuit_name) & (df_final['positionText'] == 1)].groupby('constructors.name')['constructors.name'].count().sort_values(ascending = False).index.tolist()

In [103]:
import plotly.express as px
# This dataframe has 244 lines, but 4 distinct values for `day`

fig = px.pie(values=const_wins, names=const_names,color_discrete_sequence=px.colors.sequential.RdBu)
fig.show()

In [104]:
df_final[(df_final['circuits.name'] == circuit_name) & (df_final['positionText'] == 1)].groupby('constructors.name')['constructors.name'].count().sort_values(ascending = False).tolist()

[7, 6, 2, 2, 1, 1]

In [105]:
df = df_final[(df_final['circuits.name'] == circuit_name) & (df_final['positionText'] == 1)].groupby('constructors.name')['drivers.surname'].count()

In [106]:
contructors =df_final[(df_final['circuits.name'] == circuit_name) & (df_final['positionText'] == 1)].groupby(['constructors.name', 'drivers.surname']).agg({'resultId': 'nunique'})

In [107]:
only_winners = df_final[(df_final['circuits.name'] == circuit_name) & (df_final['positionText'] == 1)]

In [108]:
only_winners

,resultId,raceId,driverId,positionText,positionOrder,races.raceId,races.year,races.circuitId,races.name,drivers.forename,...,drivers.nationality,circuits.name,circuits.location,circuits.country,circuits.lat,circuits.lng,constructors.constructorRef,constructors.name,constructors.nationality,constructorId
5519,24926,1046,815,1,1,1046,2020,3,Sakhir Grand Prix,Sergio,...,Mexican,Bahrain International Circuit,Sakhir,Bahrain,26.0325,50.5106,racing_point,Racing Point,British,211
10907,7614,4,18,1,1,4,2009,3,Bahrain Grand Prix,Jenson,...,British,Bahrain International Circuit,Sakhir,Bahrain,26.0325,50.5106,brawn,Brawn,British,23
16227,45,20,13,1,1,20,2008,3,Bahrain Grand Prix,Felipe,...,Brazilian,Bahrain International Circuit,Sakhir,Bahrain,26.0325,50.5106,ferrari,Ferrari,Italian,6
16233,413,38,13,1,1,38,2007,3,Bahrain Grand Prix,Felipe,...,Brazilian,Bahrain International Circuit,Sakhir,Bahrain,26.0325,50.5106,ferrari,Ferrari,Italian,6
16358,20323,337,4,1,1,337,2010,3,Bahrain Grand Prix,Fernando,...,Spanish,Bahrain International Circuit,Sakhir,Bahrain,26.0325,50.5106,ferrari,Ferrari,Italian,6
16373,23419,971,20,1,1,971,2017,3,Bahrain Grand Prix,Sebastian,...,German,Bahrain International Circuit,Sakhir,Bahrain,26.0325,50.5106,ferrari,Ferrari,Italian,6
16378,23802,990,20,1,1,990,2018,3,Bahrain Grand Prix,Sebastian,...,German,Bahrain International Circuit,Sakhir,Bahrain,26.0325,50.5106,ferrari,Ferrari,Italian,6
16386,25406,1074,844,1,1,1074,2022,3,Bahrain Grand Prix,Charles,...,Monegasque,Bahrain International Circuit,Sakhir,Bahrain,26.0325,50.5106,ferrari,Ferrari,Italian,6
16397,1555,92,30,1,1,92,2004,3,Bahrain Grand Prix,Michael,...,German,Bahrain International Circuit,Sakhir,Bahrain,26.0325,50.5106,ferrari,Ferrari,Italian,6
17826,743,53,4,1,1,53,2006,3,Bahrain Grand Prix,Fernando,...,Spanish,Bahrain International Circuit,Sakhir,Bahrain,26.0325,50.5106,renault,Renault,French,4


In [109]:
contructors.reset_index(inplace = True)

In [110]:
contructors

,constructors.name,drivers.surname,resultId
0,Brawn,Button,1
1,Ferrari,Alonso,1
2,Ferrari,Leclerc,1
3,Ferrari,Massa,2
4,Ferrari,Schumacher,1
5,Ferrari,Vettel,2
6,Mercedes,Hamilton,5
7,Mercedes,Rosberg,1
8,Racing Point,Pérez,1
9,Red Bull,Vettel,2


In [207]:
import plotly.express as px
df = px.data.tips()
fig = px.sunburst(contructors, path=['constructors.name', 'drivers.surname'], values='resultId',color_discrete_sequence=px.colors.sequential.RdBu)
fig.show()